In [1]:
import numpy as np
import sklearn
import os
import random
import math
import pickle

from tqdm import tqdm
from scipy.io import loadmat

np.random.seed(1)
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error


In [2]:
loaded_data = pickle.load(open("adaptive_data2.pickle", 'rb'))
X = loaded_data["X"]
Y = loaded_data["Y"]

In [3]:
data = np.concatenate([X, Y.reshape((151, 1))], axis=1)
np.random.shuffle(data)

train  = data[:50, :]
validation = data[50:100, :]
test = data[100:150, :]


X_train = train[:, 0:2]
Y_train = train[:, 2]

X_valid = validation[:, 0:2]
Y_valid = validation[:, 2]


X_test  = test[:, 0:2]
Y_test  = test[:, 2]



In [4]:
Data1 = {
    "X_train" : X_train,
    "Y_train" : Y_train,
    "X_valid" : X_valid,
    "Y_valid" : Y_valid,
    "X_test"  : X_test,
    "Y_test"  : Y_test
}

pickle.dump(Data1, open("Data/Data1.pickle", 'wb'))


# Hyperparameters

In [ ]:
Sigma_u = math.pow(10, -1.5)*np.identity(2)
C_term  = 1
Noise_variance = 1e-3

# Gaussian Process fit and predict

In [ ]:
def Kernel(x1, x2):  # K_input part of ICM kernel
    numerator    = 2*np.dot(np.dot(x1, Sigma_u), x2 )
    denominator1 = 1 + 2*np.dot(np.dot(x1, Sigma_u), x1 )
    denominator2 = 1 + 2*np.dot(np.dot(x2, Sigma_u), x2 )
    denominator  = math.sqrt(denominator1*denominator2) # See "Computing with infinite networks" for calculation of Expectation term 
    
    Expectation_term = (2/np.pi) * math.asin( numerator/denominator )
    similarity       = C_term +  Expectation_term  # See "Multitask Neural networks meet Multitask Gaussian Process" Paper for notation of C-term and Expectation term
    return similarity 




def mtgp_fit (X_train, Y_train, Noise_variance ): 
    N = len(X_train)
    K = np.zeros((N, N))
            
    for i in range(0, N):
        for j in range(0, N):
            K[i, j] = Kernel(X_train[i],  X_train[j] )
            
    B = np.zeros((N, N))
    for i in range(0, N):
        B[i, i] = Noise_variance
         
    
    C = K + B
    C_inv = np.linalg.inv(C)
    alpha = np.dot(C_inv, Y_train)
    model = {
        "X_train" :  X_train,
        "Y_train" :  Y_train,
        "Noise_variance" : Noise_variance,
        "C_inv"   :  C_inv,
        "alpha"   :  alpha,
    }
    
    return model



def mtgp_predict(X,  model):
    
    n_train  = len(model["X_train"])
    n        = len(X)
    K        = np.zeros((n_train, n))
    
    for i in range(0, n_train):
        for j in range(0, n):
            K[i, j] = Kernel( model["X_train"][i],  X[j] )
            
    y_pred = np.dot(K.T, model["alpha"] )
    return y_pred




def get_stats (y_true, y_predict):
    msr = mean_squared_error(y_true, y_predict)
    stats ={
        "msr" : msr
    }
    
    return stats



In [ ]:
model = mtgp_fit (X_train, Y_train, Noise_variance )
predict = mtgp_predict(X_valid,  model)
stats = get_stats(Y_valid, predict)
print("Validation Loss with True Hyperparameters :", stats['msr'])

Validation Loss with True Hyperparameters : 0.0012524983745783797


In [ ]:
def get_Negative_Loss (
    Sigma_u_temp, noise_variance_temp
):
    global Sigma_u
    global Noise_variance
    
    Sigma_u = math.pow(10, Sigma_u_temp)*np.identity(2)
    Noise_variance = math.pow(10, noise_variance_temp)
    
    model = mtgp_fit (X_train, Y_train, Noise_variance )
    predict = mtgp_predict(X_valid,  model)
    stats = get_stats(Y_valid, predict)
    return -stats['msr']


In [ ]:
from bayes_opt import BayesianOptimization


pbounds = {
            "Sigma_u_temp" : (-4, 2),
            "noise_variance_temp" : (-6, 0),
          }



optimizer = BayesianOptimization(
    f= get_Negative_Loss,
    pbounds=pbounds,
    random_state=1,
)


In [ ]:
optimizer.maximize(
    init_points=200,
    n_iter=100,
)

|   iter    |  target   | Sigma_... | noise_... |
-------------------------------------------------
|  1        | -0.001366 | -1.498    | -1.678    |
|  2        | -0.02753  | -3.999    | -4.186    |
|  3        | -0.006324 | -3.119    | -5.446    |
|  4        | -0.004972 | -2.882    | -3.927    |
|  5        | -0.001274 | -1.619    | -2.767    |
|  6        | -0.001285 | -1.485    | -1.889    |
|  7        | -0.02218  | -2.773    | -0.7313   |
|  8        | -0.02674  | -3.836    | -1.977    |
|  9        | -0.001224 | -1.496    | -2.648    |
|  10       | -0.005824 | -3.158    | -4.811    |
|  11       | -0.001818 |  0.8045   | -0.1904   |
|  12       | -0.003105 | -2.119    | -1.846    |
|  13       | -0.00133  |  1.258    | -0.6324   |
|  14       | -0.006341 | -3.49     | -5.766    |
|  15       | -0.02353  | -2.981    | -0.7311   |
|  16       | -0.02214  | -3.41     | -3.473    |
|  17       | -0.001657 |  1.747    | -2.801    |
|  18       | -0.001295 |  0.1513   | -4.107    |


|  103      | -0.001281 | -1.66     | -3.084    |
|  104      | -0.001251 | -0.3741   | -2.703    |
|  105      | -0.001398 |  1.557    | -0.4876   |
|  106      | -0.007202 | -1.631    | -0.2204   |
|  107      | -0.005572 | -2.956    | -5.242    |
|  108      | -0.02048  | -3.19     | -2.966    |
|  109      | -0.08294  | -3.871    | -0.3122   |
|  110      | -0.01903  |  0.9627   | -5.91     |
|  111      | -0.005039 | -2.943    | -4.008    |
|  112      | -0.02487  | -3.214    | -1.143    |
|  113      | -0.01071  | -1.932    | -0.3594   |
|  114      | -0.001741 | -0.5079   | -0.727    |
|  115      | -0.0014   |  1.068    | -0.5676   |
|  116      | -0.001225 | -1.241    | -2.722    |
|  117      | -0.001937 |  0.7916   | -4.286    |
|  118      | -0.001184 | -1.058    | -2.405    |
|  119      | -0.02748  | -3.907    | -2.439    |
|  120      | -0.001725 | -1.398    | -1.156    |
|  121      | -0.01231  | -2.109    | -0.6427   |
|  122      | -0.001297 | -0.5329   | -4.896    |


|  203      | -0.001334 |  1.999    | -0.5416   |
|  204      | -0.002193 | -2.23     | -3.642    |


In [ ]:
print(optimizer.max)

In [ ]:
Sigma_u = math.pow(10, -1.5)*np.identity(2)
C_term  = 1
Noise_variance = 1e-3



model = mtgp_fit (X_train, Y_train, Noise_variance )
predict = mtgp_predict(X_test,  model)
stats = get_stats(Y_test, predict)
print("Test Loss with True Hyperparameters : ", stats['msr'])



Sigma_u = math.pow(10, optimizer.max['params']["Sigma_u_temp"] )*np.identity(2)
C_term  = 1
Noise_variance = math.pow(10, optimizer.max['params']["noise_variance_temp"]  )


model = mtgp_fit (X_train, Y_train, Noise_variance )
predict = mtgp_predict(X_test,  model)
stats = get_stats(Y_test, predict)
print("Test Loss with BO Hyperparameters : ", stats['msr'])